### Layered feedback control circuit (sRNA mediated)


 $P_1 + I_1 \quad [d_1]<->[a_1] \quad C_1, \qquad C_1 + I_1 \quad [d_2]<->[a_2] \quad C_2 \\
    C_2 + T_1 \quad [d_3]<->[a_3]\quad  A_T, \qquad R + A_T \quad -->[k_1]\quad  T_1 \\
    A_T  -->[\beta_A] \quad M_S + A_T, \qquad M_S -->[d_M] \quad  \varnothing\\
    M_S  -->[k_{tl}]\quad  M_S + S, \qquad S -->[d_S]\quad \varnothing  \\
    S  -->[k_r]\quad  S^*, \qquad S^* -->[d_{S^*}] \quad \varnothing \\
     S^* + I_2 \quad [d_4]<->[a_4]\quad  C_C, \qquad C_C + T_2 \quad [d_5]<->[a_5]\quad  A_C \\
    R + A_C \quad -->[k_2]\quad  T_2 \\
    A_C  -->[\beta_C]\quad R + M_G + A_C, \qquad R -->[d_R]\quad  \varnothing\\
    M_G  -->[k_{G}]\quad  M_G + G, \qquad G -->[d_G]\quad\varnothing \\
    G  -->[k_{r_2}] \quad G^*, \qquad G^* -->[d_{G^*}]\quad \varnothing $

In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from auto_reduce import *
import numpy as np
from sympy import symbols

In [2]:
# Post conservation law and other approximations phenomenological model at the RNA level
n = 17 # Number of states : P, A_T, C1, T1, R, C2, M_S, S, Ss, I2, C_C, A_C, M_G, G, Gs, T2, I1
nouts = 1 # Number of outputs, Gs

# Inputs by user 
x_init = np.zeros(n)
x_init[0] = 10
x_init[1] = 10000
x_init[4] = 10000
x_init[6] = 10000
C = np.zeros((nouts,n), dtype=int)
C[0][14] = 1 #Gs

error_tol = 30000
# System dynamics symbolically

P1 = symbols('P1')
A_T = symbols('A_T')
C1 = symbols('C1') # P:I1
T1 = symbols('T1')
R = symbols('R')
C2 = symbols('C2') # C1:I1
M_S = symbols('M_S')
S = symbols('S') 
Ss = symbols('Ss')

I2 = symbols('I2')
C_C = symbols('C_C')
A_C = symbols('A_C') 
M_G = symbols('M_G')
G = symbols('G')
Gs = symbols('Gs') 
T2 = symbols('T2')
I1 = symbols('I1')
# x = [x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16]
x = [P1, A_T, C1, T1, R, C2, M_S, S, Ss, I2, C_C, A_C, M_G, G, Gs, T2, I1]


a1 = symbols('a1')
d1 = symbols('d1')
a2 = symbols('a2')
d2 = symbols('d2')
a3 = symbols('a3')
d3 = symbols('d3')
k1 = symbols('k1')
b_A = symbols('b_A')
d_M = symbols('d_M')
k_tl = symbols('k_tl')
d_S = symbols('d_S')
k_r = symbols('k_r')
d_Ss = symbols('d_Ss')
a4 = symbols('a4')
d4 = symbols('d4')
a5 = symbols('a5')
d5 = symbols('d5')
k2 = symbols('k2')
b_C = symbols('b_C')
d_R = symbols('d_R')
k_G = symbols('k_G')
d_G = symbols('d_G')
d_Gs = symbols('d_Gs')
k_r2 = symbols('k_r2')


T1_tot = symbols('T1_tot')
T2_tot = symbols('T2_tot')
# R_tot = symbols('R_tot')

params = [a1,d1,a2,d2,a3,d3,k1,b_A,d_M,k_tl,d_S,k_r,d_Ss,a4,d4,a5,d5,k2,b_C,d_R,k_G,d_G,d_Gs,k_r2,T1_tot,T2_tot]

# Number of states : 
# P (0), A_T (1), C1 (2), T1 (3), R (4), C2 (5), M_S (6), S (7), Ss (8), I2 (9), 
# C_C (10), A_C (11), M_G (12), G (13), Gs (14), T2 (15), I1 (16)
f0 = d1*C1 - a1*P1*I1 #P
f1 = a3*C2*T1 - d3*A_T - k1*R*A_T  #A_T
f2 = a1*P1*I1 + d2*C2 - d1*C1 - a2*C1*I1 #C1
f3 = d3*A_T - a3*C2*T1 + k1*R*A_T #T1
f4 = b_C*A_C - k1*R*A_T - k2*R*A_C - d_R*R #R
f5 = a2*C1*I1 + d3*A_T - d2*C2 - a3*C2*T1 #C2
f6 = b_A*A_T - d_M*M_S #M_S
f7 = k_tl*M_S - d_S*S - k_r*S #S
f8 = k_r*S + d4*C_C - d_Ss*Ss - a4*Ss*I2 # Ss
f9 = d4*C_C - a4*Ss*I2 #I_2
f10 = a4*Ss*I2 + d5*A_C - d4*C_C - a5*C_C*T2 #C_C
f11 = a5*C_C*T2 - d5*A_C - k2*R*A_C #A_C
f12 = b_C*A_C - d_M*M_G #M_G
f13 = k_G*M_G - d_G*G - k_r2*G #G
f14 = k_r2*G - d_Gs*Gs #Gs
f15 = d5*A_C + k2*R*A_C - a5*C_C*T2#T2
f16 = d1*C1 + d2*C2 - a1*P1*I1 - a2*C1*I1 #I1

f = [f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14, f15, f16]
# parameter values
params_values = [100, 10, 4, 10, 0.25, 2, 10, 0.5, 1, 1, 1000, 1000, 1000]
sys = System(x, f, params = params, params_values = params_values, C = C, x_init = x_init)

In [3]:
from auto_reduce.utils import reduce
timepoints_ssm = np.linspace(0,20,10)
timepoints_ode = np.linspace(0,20,10)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)
sys_reduce.nstates_tol = 8

In [4]:
conserved_quantities = [A_T + T1 - T1_tot, A_C + T2 - T2_tot]
states_to_eliminate = [1,11]
f_cons = sys_reduce.set_conservation_laws(conserved_quantities, states_to_eliminate)

In [5]:
#  P1 (0), C1 (1), T1 (2), R (3), C2 (4), M_S (5), S (6), Ss (7), 
# I2 (8), C_C (9), M_G (10), G (11), Gs (12), T2 (13), I1 (14)

# We want to retain R, M_S, C_C, M_G, G, Gs, I1(input)
reduced_sys, fast_ss = sys_reduce.solve_timescale_separation([3,5,9,10,11,12,14])


attempting : [3, 5, 9, 10, 11, 12, 14]
Reduced set of variables is [R, M_S, C_C, M_G, G, Gs, I1]
f_hat =  [-R*d_R - R*k1*(-T1 + T1_tot) - R*k2*(-T2 + T2_tot) + b_C*(-T2 + T2_tot), -M_S*d_M + b_A*(-T1 + T1_tot), -C_C*T2*a5 - C_C*d4 + I2*Ss*a4 + d5*(-T2 + T2_tot), -M_G*d_M + b_C*(-T2 + T2_tot), -G*d_G - G*k_r2 + M_G*k_G, G*k_r2 - Gs*d_Gs, -C1*I1*a2 + C1*d1 + C2*d2 - I1*P1*a1]
Collapsed set of variables is [P1, C1, T1, C2, S, Ss, I2, T2]
Solved for P1 to get [C1*d1/(I1*a1)]
Solved for C1 to get [(C2*d2 + I1*P1*a1)/(I1*a2 + d1)]
Solved for T1 to get [T1_tot*(R*k1 + d3)/(C2*a3 + R*k1 + d3)]
Solved for C2 to get [(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2)]
Solved for S to get [M_S*k_tl/(d_S + k_r)]
Solved for Ss to get [(C_C*d4 + S*k_r)/(I2*a4 + d_Ss)]
Solved for I2 to get [C_C*d4/(Ss*a4)]
Solved for T2 to get [T2_tot*(R*k2 + d5)/(C_C*a5 + R*k2 + d5)]
Substituting C1*d1/(I1*a1) for variable P1 into f_hat0
f_hat =  -R*d_R - R*k1*(-T1 + T1_tot) - R*k2*(-T2 + T2_tot) + b_C*(-T2 + T2_tot)
Subst

f_hat =  -R*d_R - R*k1*(-T1_tot*(R*k1 + d3)/(R*k1 + a3*(I1*a2*(I1*P1*a1 + d2*(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2))/(I1*a2 + d1) - T1_tot*d3*(R*k1 + d3)/(R*k1 + a3*(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2) + d3) + T1_tot*d3)/(T1_tot*a3*(R*k1 + d3)/(R*k1 + a3*(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2) + d3) + d2) + d3) + T1_tot) - R*k2*(-T2_tot*(R*k2 + d5)/(C_C*a5 + R*k2 + d5) + T2_tot) + b_C*(-T2_tot*(R*k2 + d5)/(C_C*a5 + R*k2 + d5) + T2_tot)
Substituting (C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2) for variable C2 into f_hat1
f_hat =  -M_S*d_M + b_A*(-T1_tot*(R*k1 + d3)/(R*k1 + a3*(I1*a2*(I1*P1*a1 + d2*(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2))/(I1*a2 + d1) - T1_tot*d3*(R*k1 + d3)/(R*k1 + a3*(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2) + d3) + T1_tot*d3)/(T1_tot*a3*(R*k1 + d3)/(R*k1 + a3*(C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2) + d3) + d2) + d3) + T1_tot)
Substituting (C1*I1*a2 - T1*d3 + T1_tot*d3)/(T1*a3 + d2) for variable C2 into f_hat2
f_hat =  -C_C*T2_tot*a5*(R*k2 + d5

C:\Users\apand\OneDrive\Documents\synBio\autoReduce\auto_reduce\model_reduction.py:300: UserWarning: Check model consistency
  warnings.warn('Check model consistency')


In [6]:
f_cons

[C1*d1 - I1*P1*a1,
 -C1*I1*a2 - C1*d1 + C2*d2 + I1*P1*a1,
 -C2*T1*a3 + R*k1*(-T1 + T1_tot) + d3*(-T1 + T1_tot),
 -R*d_R - R*k1*(-T1 + T1_tot) - R*k2*(-T2 + T2_tot) + b_C*(-T2 + T2_tot),
 C1*I1*a2 - C2*T1*a3 - C2*d2 + d3*(-T1 + T1_tot),
 -M_S*d_M + b_A*(-T1 + T1_tot),
 M_S*k_tl - S*d_S - S*k_r,
 C_C*d4 - I2*Ss*a4 + S*k_r - Ss*d_Ss,
 C_C*d4 - I2*Ss*a4,
 -C_C*T2*a5 - C_C*d4 + I2*Ss*a4 + d5*(-T2 + T2_tot),
 -M_G*d_M + b_C*(-T2 + T2_tot),
 -G*d_G - G*k_r2 + M_G*k_G,
 G*k_r2 - Gs*d_Gs,
 -C_C*T2*a5 + R*k2*(-T2 + T2_tot) + d5*(-T2 + T2_tot),
 -C1*I1*a2 + C1*d1 + C2*d2 - I1*P1*a1]

In [7]:
reduced_sys.x

[R, M_S, C_C, M_G, G, Gs, I1]

In [8]:
#A_C, A_T, Ss, S
sys_reduce.x

[P1, C1, T1, R, C2, M_S, S, Ss, I2, C_C, M_G, G, Gs, T2, I1]